In [1]:
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.env_util import make_vec_env

from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

from gymnasium.envs.registration import register

from gymnasium import spaces
import torch as th
from torch import nn
import numpy as np
 
import gymnasium as gym
import os

from gym_data.envs import CarcassoneEnv
from stable_baselines3.common.vec_env import SubprocVecEnv


/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:216: UserWarning: WARN: Overriding environment ALE/Adventure-v5
  logger.warn("Overriding environment {}".format(id))
/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:216: UserWarning: WARN: Overriding environment ALE/Adventure-ram-v5
  logger.warn("Overriding environment {}".format(id))
/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:216: UserWarning: WARN: Overriding environment ALE/AirRaid-v5
  logger.warn("Overriding environment {}".format(id))
/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:216: UserWarning: WARN: Overriding environment ALE/AirRaid-ram-v5
  logger.warn("Overriding environment {}".format(id))
/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:216: UserWarning: WARN: Overriding environment ALE/Alien-v5
  logger.warn("Overriding environment {}".format(id))
/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:216: UserWarning: WA

In [2]:
import stable_baselines3

In [3]:
def linear_schedule(initial_value,total_steps):
    """
    Linear learning rate schedule.
    :param initial_value: (float or str)
    :return: (function)
    """
    if isinstance(initial_value, str):
        initial_value = float(initial_value)
    final_value = 1e-9
    last_progress = 0.7

    def func(progress):
        """
        Progress will decrease from 1 (beginning) to 0
        :param progress: (float)
        :return: (float)
        """
        if progress <= last_progress:
            return final_value
        else:
            return initial_value * ( progress * ( 1 - last_progress ) + last_progress )

    return func
    

In [4]:
env = make_vec_env(CarcassoneEnv, seed=42,  n_envs=32, vec_env_cls=SubprocVecEnv)

/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:216: UserWarning: WARN: Overriding environment ALE/Adventure-v5
  logger.warn("Overriding environment {}".format(id))
/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:216: UserWarning: WARN: Overriding environment ALE/Adventure-ram-v5
  logger.warn("Overriding environment {}".format(id))
/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:216: UserWarning: WARN: Overriding environment ALE/AirRaid-v5
  logger.warn("Overriding environment {}".format(id))
/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:216: UserWarning: WARN: Overriding environment ALE/AirRaid-ram-v5
  logger.warn("Overriding environment {}".format(id))
/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:216: UserWarning: WARN: Overriding environment ALE/Alien-v5
  logger.warn("Overriding environment {}".format(id))
/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:216: UserWarning: WA

In [5]:
total_timesteps = 6_000_000_000
from stable_baselines3.common.callbacks import BaseCallback

env.env_method("set_random", int(0))  

for i in range(32):
    env.env_method("connect_ws",indices=i, kwargs={"i": i})                                     
        
class TimestepCallback(BaseCallback):
    def __init__(self, verbose=0, vec_env=None):
        super(TimestepCallback, self).__init__(verbose=verbose)
        self.vec_env = vec_env

    def _on_step(self):
        # Get the current number of timesteps completed
        num_timesteps = self.num_timesteps
        if num_timesteps % 10000 == 0 and num_timesteps != 0:
            # Call the updateProgress function on the VecEnv
            if self.vec_env is not None:
                self.vec_env.env_method("updateProgress",( 1 / 100 + num_timesteps / total_timesteps ))


In [6]:
# env = make_vec_env(make_env,seed=1, n_envs=32)
from sb3_contrib import MaskablePPO
from stable_baselines3.common.callbacks import EveryNTimesteps

callback = TimestepCallback(vec_env=env)

event_callback = EveryNTimesteps(n_steps=10000, callback=callback)

# model = PPO("MultiInputPolicy", env, verbose=0,tensorboard_log="runs/new_run", device="auto", ent_coef=0.1, learning_rate=linear_schedule(0.0001)) #, policy_kwargs=policy_kwargs)
model = PPO.load("model_carcassone_curr_lern3", env, verbose=0, n_epochs=10 ,batch_size=64,tensorboard_log="runs/curr_lern",device="cuda", n_steps=2048, ent_coef=0.001, clip_range=0.2, learning_rate=linear_schedule(5e-7, total_timesteps)) #, policy_kwargs=policy_kwargs)
# model = A2C.load("carcassone_model_A2C_1", env, tensorboard_log="runs/A2C_7", device="auto", n_steps=64)#, learning_rate= 0.0001)
# model.set_parameters("model_carcassone")



Total concat size:  10304


In [7]:
try:
    model.learn(total_timesteps=total_timesteps, callback=event_callback)
except KeyboardInterrupt:
    
    interrupted = True

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
Task was destroyed but it is pending!
task: <Task pending name='Task-2' coro=<WebSocketCommonProtocol.transfer_data() running at /opt/conda/lib/python3.10/site-packages/websockets/legacy/protocol.py:974> wait_for=<Future pending cb=[shield.<locals>._outer_done_callback() at /opt/conda/lib/python3.10/asyncio/tasks.py:864, Task.task_wakeup()]> cb=[Task.task_wakeup()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-4' coro=<WebSocketCommonProtocol.close_connection() running at /opt/conda/lib/python3.10/site-packages/websockets/legacy/protocol.py:1301> wait_for=<Task pending name='Task-2' coro=<WebSocketCommonProtocol.transfer_data() running at /opt/conda/lib/python3.10/site-packages/websockets/legacy/protocol.py

WebSocket connected
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection clo

EOFError: 

In [ ]:
# model.save("model_carcassone_curr_lern4")

In [ ]:
# env = make_vec_env(CarcassoneEnv, seed=42,  n_envs=1, vec_env_cls=SubprocVecEnv)

In [ ]:
# done = False
# obs = env.reset()
# rewards = []

# model = PPO("MultiInputPolicy", env, verbose=1,tensorboard_log="runs/", device="cuda", n_steps=1024, ent_coef=0.04, learning_rate=linear_schedule(0.0003), policy_kwargs=policy_kwargs)

# model = PPO.load("model_carcassone_big2")

In [ ]:
# sim_rewards = []
# n_iter = 1000
# for i in range(n_iter):
#     done = False
#     obs = env.reset()
#     rewards = []
    
#     while not done:
        
#         action, _state = model.predict(obs, deterministic=False)
#         obs, reward, done, info = env.step(action)
#         sim_rewards.append(reward[0])
        

In [ ]:
# print(sum(sim_rewards) // n_iter)
# print(len([x for x in sim_rewards if x < 0]) // n_iter)
# print(len([x for x in sim_rewards if x > 0]) // n_iter)

Process ForkServerProcess-32:
Process ForkServerProcess-28:
Process ForkServerProcess-24:
Process ForkServerProcess-31:
Process ForkServerProcess-26:
Process ForkServerProcess-20:
Process ForkServerProcess-18:
Process ForkServerProcess-22:
Process ForkServerProcess-10:
Process ForkServerProcess-27:
Process ForkServerProcess-25:
Process ForkServerProcess-15:
Process ForkServerProcess-16:
Process ForkServerProcess-19:
Process ForkServerProcess-17:
Process ForkServerProcess-29:
Process ForkServerProcess-23:
Process ForkServerProcess-4:
Process ForkServerProcess-6:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (m

WebSocket connected
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebSocket connected
WebSocket connection closed unexpectedly with code 1006: 
WebSocket connected
WebS

Para poder ir obteniendo resultados gradualmente se realizó:
- 1. Primero entrenarlo únicamente con semilla aleatoria de 0 de forma que se pueda ver si realmente está entrenando o no.  PPO_1
- 2. Después para ver cómo se comportava con diferentes acciones del oponente se añadió la semilla aleatoria a las acciones del oponente PPO_4
- 3. Después se añadió esta aleatoriedad también a la partida. En ambos casos tenían 1 millón de pasos de entrenamiento para acostumbrarse a la nueva situación PPO_13
- 4. Después en cambio de subir de 1 en 1 las posibilidades aleatorias cada millón de pasos se subió de 10 posibilidades aleatorias cada 10 millones de pasos
- 5. Finalmente se provó quitando toda la gestión de aleatoriedad de forma que fuese como una partida normal i corriente